In [3]:
import geopandas as gpd
import pandas as pd
from copy import deepcopy
from geopandas import GeoDataFrame
from shapely.ops import unary_union

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
gdf_center = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/国土数値情報/土地利用メッシュ/gdf_center.shp')

In [16]:
gdf_station = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/国土数値情報/鉄道/N02-22_Station.shp', encoding='shift-jis')

In [7]:
#座標系を変更
src_proj = 4326 # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定

# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_center.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_center = gdf_center.to_crs(epsg=dst_proj)  # 変換後座標に変換

gdf_station.crs = f'epsg:{src_proj}'
gdf_station = gdf_station.to_crs(epsg=dst_proj)

In [27]:
#線路のデータフレームのカラム名を変換
station_columns = {'N02_001': 'station_classification',
                   'N02_002': 'station_Business_Type',
                   'N02_003': 'station_route_name',
                   'N02_004':'station_operating_company',
                   'N02_005':	'station_name',
                   'N02_005c': 'station_code',
                   'N02_005g': 'station_group_code'}
gdf_station.rename(columns=station_columns, inplace=True)

In [10]:
#空間結合をする
result = gpd.sjoin_nearest(gdf_center, gdf_station, how="left",lsuffix='point',rsuffix='road')
#空間結合された道路のgeometryを追加
result = pd.merge(result, gdf_station[['station_name', 'geometry']], on='station_name')
#geometryの名前を変更
result.rename(columns={'geometry_y':'nearest_station'}, inplace=True)
#メモリの解放
del gdf_station

In [12]:
#dictance_railwyカラムを追加
result['distance_station'] = None

for i in range((len(result) // 50)+1):
    try:
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_station'][50*i:50*(i+1)]))

        # ポイントとマルチラインストリングとの最短距離を計算
        result['distance_station'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
        #一回ごとにメモリを解放
        del merged_lines
    except Exception as e:
        # エラーが発生した場合の処理
        print(f"Error occurred at the last batch: {e}")
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_station'][50*(i+1):]))

        result['distance_station'][50*(i+1):] = result.geometry_x[50*(i+1):].apply(lambda p: p.distance(merged_lines))

ストリーミング出力は最後の 5000 行に切り捨てられました。
<ipython-input-12-3ad9af224296>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_station'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-12-3ad9af224296>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_station'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-12-3ad9af224296>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [14]:
#重複を削除
result = result[~result.Meshcode.duplicated()]

#indexを並び替え
result.reset_index(inplace=True)

In [41]:
result_columns = result.columns.to_list()[6:]
result_columns[:0] = ['Meshcode']
result[result_columns].to_csv('/content/drive/Shareddrives/geomap/周辺環境データ構築/dev/奥州市/奥州市　最近傍鉄道/奥州市_最近傍駅.csv', encoding='shift-jis', index=False)